In [189]:
import numpy as np
from random import randint, shuffle, random

In [190]:
# Génération de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(1, 10)
            if i == j:
                cout = 0
            k = i-1 if i-1 > 0 else n_ville
            if (i+1)%n_ville == j or k == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout

    return mat
 

In [191]:
#mat=[[0,2,1],[2,0,1],[1,2,0]]
#population=[]


# Création de la population initial, de taille size
def population_init(mat, size):
    population = []
    
    for _ in range(0, size):
        circuit = list(range(0, len(mat[0])))
        shuffle(circuit) # circuit random
        population.append([circuit,5])
        
    return population

#population_init(mat, 5)

In [192]:
#mat=[ [0,5,2], [4,0,3], [1,5,0] ]

def fitness(individu, mat):
    sum = 0
    
    for i in range(len(individu)):
        sum += mat[individu[i]][individu[(i+1) % len(individu)]] 
        
    return sum

#fitness([1,0,2], mat)

In [193]:
def mort(pop):
    population = []
    
    for i in range(len(pop)):
        pop[i][1] = pop[i][1]-1
        
        if pop[i][1] != 0:
            population.append(pop[i])
            
    return population

In [194]:
def selection(population, mat):
    new_pop = []
    sum = 0
    # Somme des fitness
    for p in population:
        sum = sum + fitness(p[0], mat) 
    
    for p in population:
        f = fitness(p[0], mat)
        if random() > f / sum: 
            new_pop.append(p)
            
            
    return new_pop
    
        


In [195]:
def croisement(population, nb_enfants):
    for _ in range(0, nb_enfants // 2):
        #selection des parents
        indice_parent1 = randint(0, len(population)-1)
        indice_parent2 = randint(0, len(population)-1)
        #print("indice du parent1:",indice_parent1,"  indice du parent 2:",indice_parent2)
        #print("taille de la pop:",len(population)-1)

        parent1=population[indice_parent1][0]
        parent2=population[indice_parent2][0]
        indice_same=randint(1, len(parent1) - 1)
        enfant1 = []
        enfant2 = []
        enfant1_temp = []
        enfant2_temp = []

        for x in range(0, indice_same):
            enfant1_temp.append(parent1[x])
            enfant2_temp.append(parent2[x])

        for y in range(indice_same, len(parent1)):
            enfant1_temp.append(parent2[y])
            enfant2_temp.append(parent1[y])

        for a in range(0, len(enfant1_temp)):
            if enfant1_temp[a] not in enfant1:
                enfant1.append(enfant1_temp[a])
            if enfant2_temp[a] not in enfant2:
                enfant2.append(enfant2_temp[a])
        
        for b in range(0, len(parent1)):
            if b not in enfant1:
                enfant1.append(b)
            if b not in enfant2:
                enfant2.append(b)
        
        population.append([enfant1,5])
        population.append([enfant2,5])
        #print(population)

#pop=[[[0,1,2,3],5],[[1,2,3,0],5]]
#croisement_v2(pop)
#print(pop)

In [196]:
def mutation(pop):
    mut=[]
    while(len(mut) != 3):
        pos = randint(0, len(pop)-1)
        mut.append(pos)
        
    for i in mut:
        perm1 = randint(0, len(pop[i][0])-1)
        perm2 = randint(0, len(pop[i][0])-1)
        #print(pop[i],"->",perm1,"   ",perm2)
        val1 = pop[i][0][perm1]
        val2 = pop[i][0][perm2]
        temp = val2
        pop[i][0][perm2]=val1
        pop[i][0][perm1]=temp
        #temp=pop[i][0][perm2]
        #perm2=pop[i][0][perm1]
        #perm1=temp
#pop=[[[0,1,2,3],5],[[4,5,6,7],5],[[8,9,1,2],5],[[3,4,5,6],5]]
#mutation(pop)
#print(pop)

In [197]:
def verif(pop, valeur, chemin, mat):
    for i in range(len(pop)):
        if fitness(pop[i][0], mat) < valeur:
            valeur = fitness(pop[i][0], mat)
            chemin = pop[i][0]
    return valeur, chemin
#mat=[[0,3,4,2],[5,0,6,1],[4,6,0,3],[2,1,3,0]]
#pop=[[[0,1,2,3],5],[[3,1,2,0],5],[[0,3,1,2],5],[[3,2,1,0],5]]
#val,chemin=verif(pop,999)
#print(val,"  ",chemin)

In [199]:
def main_algo_genetique():
    val_min = float("inf")
    chemin_min = []
    time = 0
    time_max = 5000
    mat = create_matrix(12)
    population = population_init(mat, 5) 
    population = selection(population, mat)
    val_min,chemin_min = verif(population, val_min, chemin_min, mat)
    print("iteration:", time, ", nb individu:", len(population))

    while time < time_max and len(population) >= 2:
        time += 1 
        croisement(population, 6)
        mutation(population)
        population = selection(population, mat)
        population = mort(population)
        val_min, chemin_min = verif(population, val_min, chemin_min, mat)
        print("iteration:", time, ", nb individu:", len(population), "val_min: ", val_min) 
        
    print("Chemin min: ", chemin_min, " pour un cout de: ", val_min)
    
main_algo_genetique()

iteration: 0 , nb individu: 5
iteration: 1 , nb individu: 11 val_min:  37.0
iteration: 2 , nb individu: 16 val_min:  35.0
iteration: 3 , nb individu: 22 val_min:  35.0
iteration: 4 , nb individu: 27 val_min:  35.0
iteration: 5 , nb individu: 23 val_min:  35.0
iteration: 6 , nb individu: 24 val_min:  35.0
iteration: 7 , nb individu: 24 val_min:  35.0
iteration: 8 , nb individu: 24 val_min:  35.0
iteration: 9 , nb individu: 23 val_min:  35.0
iteration: 10 , nb individu: 22 val_min:  35.0
iteration: 11 , nb individu: 22 val_min:  35.0
iteration: 12 , nb individu: 23 val_min:  35.0
iteration: 13 , nb individu: 22 val_min:  35.0
iteration: 14 , nb individu: 20 val_min:  35.0
iteration: 15 , nb individu: 21 val_min:  35.0
iteration: 16 , nb individu: 20 val_min:  35.0
iteration: 17 , nb individu: 22 val_min:  35.0
iteration: 18 , nb individu: 23 val_min:  35.0
iteration: 19 , nb individu: 21 val_min:  35.0
iteration: 20 , nb individu: 22 val_min:  35.0
iteration: 21 , nb individu: 23 val_min